In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from utils.vino_functions import PrepareNetWork,PrepareInputImage,MakePrediction
from pathlib import Path
import time

## Read In Videos

In [2]:
# Model Files
model_root = Path('model/intel/person-vehicle-bike-detection-crossroad-1016/FP16-INT8')
model_bin = model_root / 'person-vehicle-bike-detection-crossroad-1016.bin'
model_xml = model_root / 'person-vehicle-bike-detection-crossroad-1016.xml'
device = 'CPU'

# Video Path
video = 'data/15trim_highway_traffic.mp4'

# Labels
labels_map = {0:'non-vehicle',
              1:'vehicle',
              2:'person'}

# Prepare Model
car_detection_names, car_detection_exec_net, car_detection_img_shape = PrepareNetWork(model_xml,model_bin,device)

Extract Model Input Dimension: 1 3 512 512


In [3]:
## [image_id, label, conf, x_min, y_min, x_max, y_max]

def detection_box(result, output_key, prob_thresh, image,labels_map):
    res = result[output_key].reshape(200,7)
    height,width = image.shape[:2]
    
    detector = []
    
    
    for obj in res:
        conf = obj[2]
        # Get Class ID
        class_id = int(obj[1])
        
        if (conf >= prob_thresh) & (class_id in labels_map):
            
            # get coordinates
            xmin = int(obj[3] * width)
            ymin = int(obj[4] * height)
            xmax = int(obj[5] * width)
            ymax = int(obj[6] * height)
            

            
            
            # Draw Centroid
            c_x = int((xmin+xmax) / 2)
            c_y = int((ymin+ymax) / 2)
            
            detector.append((c_x,c_y))

    return detector

In [6]:
# Video Reading utility 
cap = cv2.VideoCapture(video)
car_cnt = 0

# frame_name = 0
while cap.isOpened():

    
    ret, frame = cap.read()
#     frame_name += 1 
    
    if not ret:
        # No Frames to render
        break
    
    start_time = time.time()
    
    image = frame[200:,640:]

    
    _, in_frame = PrepareInputImage(image,car_detection_img_shape)
    result = MakePrediction(car_detection_exec_net,car_detection_names[0],in_frame)
    detector = detection_box(result,car_detection_names[1], 0.7, image, labels_map)
#     print(detector)
    
    roi_pt1 = (100,100)
    roi_pt2 = (600,107)
    

    tracker_n = []
    for id,val in enumerate(detector):
        c_x,c_y = val
        if roi_pt1[1] <= c_y <= roi_pt2[1]:
            tracker_n.append((id,val))

    for item in tracker_n:
        id, pos = item


        color = (0,255,125)
        cv2.circle(image,pos,5,color,-1)
        cv2.putText(image,text=f'{id}',org = (pos[0],pos[1]-10),
                    fontFace = cv2.FONT_HERSHEY_SIMPLEX,
                    fontScale = 0.5, color = (0,0,255),
                    thickness = 2, lineType = cv2.LINE_AA)

        car_cnt+=1

    
    
    
    end_time = time.time()
    fps = np.round(1/(end_time-start_time))
    cv2.putText(image,text = f"Current FPS: {fps}",org = (0,30),
                fontFace = cv2.FONT_HERSHEY_SIMPLEX,
                fontScale = 0.5, color = (0,0,255),
                thickness = 2, lineType = cv2.LINE_AA)
    
    cv2.putText(image,text = f"Car Count: {car_cnt}",org = (0,60),
                fontFace = cv2.FONT_HERSHEY_SIMPLEX,
                fontScale = 0.5, color = (0,255,255),
                thickness = 2, lineType = cv2.LINE_AA)
    
    cv2.rectangle(image, roi_pt1, roi_pt2,(0,0,255),1)
    
    cv2.imshow('traffic', image)
    
#     cv2.imwrite(f"frames/{str(frame_name).zfill(5)}.png",image)

    if cv2.waitKey(30) == ord('q'):
        break

        
## Very Important to remeber to close all windows to avoid breakdown
cap.release()
cv2.destroyAllWindows()
